<a href="https://colab.research.google.com/github/Warpedro1/Flight-delay-predction/blob/main/flight_delays_competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [ ]:
import pandas as pd
from google.colab import files
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import cross_val_score

from sklearn.metrics import roc_auc_score
from sklearn.tree import export_graphviz
import graphviz

!pip install catboost
from catboost import CatBoostClassifier

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import FunctionTransformer, Normalizer

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import TransformerMixin, BaseEstimator

!pip install eli5
from eli5.catboost import explain_weights

uploaded = files.upload()

train_ = pd.read_csv('flight_delays_train.csv')
test_ = pd.read_csv('flight_delays_test.csv')
print(train_.shape)

Saving flight_delays_test.csv to flight_delays_test.csv
Saving flight_delays_train.csv to flight_delays_train.csv
(100000, 9)


In [ ]:
SEED = 42

#Análise

In [ ]:
train_.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [ ]:
train_.describe()

,DepTime,Distance
count,100000.000000,100000.00000
mean,1341.523880,729.39716
std,476.378445,574.61686
min,1.000000,30.00000
25%,931.000000,317.00000
50%,1330.000000,575.00000
75%,1733.000000,957.00000
max,2534.000000,4962.00000


In [ ]:
train_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column           Non-Null Count   Dtype   
---  ------           --------------   -----   
 0   Month            100000 non-null  object  
 1   DayofMonth       100000 non-null  object  
 2   DayOfWeek        100000 non-null  object  
 3   DepTime          100000 non-null  int64   
 4   UniqueCarrier    100000 non-null  object  
 5   Origin           100000 non-null  object  
 6   Dest             100000 non-null  object  
 7   Distance         100000 non-null  int64   
 8   Y                100000 non-null  int64   
 9   DepTimeCategory  99983 non-null   category
 10  DayOfWeekRank    0 non-null       float64 
 11  MonthRank        100000 non-null  int64   
dtypes: category(1), float64(1), int64(4), object(6)
memory usage: 8.5+ MB


In [ ]:
train_.isna().sum()

Month                0
DayofMonth           0
DayOfWeek            0
DepTime              0
UniqueCarrier        0
Origin               0
Dest                 0
Distance             0
dep_delayed_15min    0
dtype: int64

In [ ]:
for feature in train_:
  class_count = train_[feature].value_counts(normalize=True)
  print(class_count)

In [ ]:
sns.pairplot(train_.select_dtypes(include=[np.number]))
plt.figure(figsize=(48, 32))
plt.show()

In [ ]:
for column in train_.columns:
    plt.hist(train_[column])
    plt.title(f'Histogram of {column}')
    plt.xlabel('Values')
    plt.ylabel('Frequency')
    plt.show()

In [ ]:
train_['Origin'].nunique()

289

In [ ]:
import plotly.express as px


In [ ]:
pecas = train_.loc[train_['Distance']]



fig = px.box(train_['Distance'])
fig.update_layout(title='Boxplot método Peças - Notas  Antes x  Notas Depois')
fig.show()

"Route category lengths tend to define short-haul routes as being shorter than 600–800 nmi (1,100–1,500 km), long-haul as being longer than 2,200–2,600 nmi (4,100–4,800 km), and medium-haul as being in-between."

#Tratamento

In [ ]:
train_.rename({'dep_delayed_15min': 'Y'}, axis = 1,inplace =True)
train_['Y'].replace({'N': 0, 'Y':1}, inplace = True)

In [ ]:
train_['DepTime'] = train_['DepTime'].astype(int)

In [ ]:
df_onehot = pd.get_dummies(train_, columns=['Origin', 'Dest','UniqueCarrier'])
df_onehot['Month'].replace()

0         c-8
1         c-4
2         c-9
3        c-11
4        c-10
         ... 
99995     c-5
99996     c-1
99997     c-1
99998     c-4
99999    c-11
Name: Month, Length: 100000, dtype: object

In [ ]:
df_onehot['Month'] = df_onehot['Month'].str.split('-').str[-1]
df_onehot['Month'] = df_onehot['Month'].astype(int)

df_onehot['DayofMonth'] = df_onehot['DayofMonth'].str.split('-').str[-1]
df_onehot['DayofMonth'] = df_onehot['DayofMonth'].astype(int)

df_onehot['DayOfWeek'] = df_onehot['DayOfWeek'].str.split('-').str[-1]
df_onehot['DayOfWeek'] = df_onehot['DayOfWeek'].astype(int)

df_onehot['DepTime'] = df_onehot['DepTime'].astype(int)



In [ ]:
df_onehot['Y'].replace({'N': 0, 'Y':1}, inplace = True)

In [ ]:
df_onehot.head()

,Month,DayofMonth,DayOfWeek,DepTime,Distance,Y,Origin_ABE,Origin_ABI,Origin_ABQ,Origin_ABY,...,UniqueCarrier_MQ,UniqueCarrier_NW,UniqueCarrier_OH,UniqueCarrier_OO,UniqueCarrier_TZ,UniqueCarrier_UA,UniqueCarrier_US,UniqueCarrier_WN,UniqueCarrier_XE,UniqueCarrier_YV
0,8,21,7,1934,732,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,20,3,1548,834,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,9,2,5,1422,416,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,11,25,6,1015,872,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,10,7,6,1828,423,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


##Tratamento para o catboost

In [ ]:
train_cat = train_.copy()

train_cat['R'] = train_cat['Origin'] + "_" + train_cat['Dest']

train_cat['DepTimeCategory'] = pd.cut(train_cat['DepTime'], bins=[0,500,1200,1700,2100,2400], labels=['Night','Morning', 'Afternoon', 'Evening','Night'], ordered=False)

train_cat['DistanceCategory'] = pd.cut(train_cat['Distance'], bins=[0,1500,4100,5000], labels=['Short-haul','Medium-haul' ,'Long-haul' ])

train_cat.dropna(inplace=True)

train_cat['Month'] = train_cat['Month'].str.split('-').str[-1]
train_cat['Month'] = train_cat['Month'].astype(int)

train_cat['DayofMonth'] = train_cat['DayofMonth'].str.split('-').str[-1]
train_cat['DayofMonth'] = train_cat['DayofMonth'].astype(int)

train_cat['DayOfWeek'] = train_cat['DayOfWeek'].str.split('-').str[-1]
train_cat['DayOfWeek'] = train_cat['DayOfWeek'].astype(int)

train_cat['Carrier+Origin'] = train_cat['UniqueCarrier'] + "_" + train_cat['Origin']
train_cat['Carrier+Dest'] = train_cat['UniqueCarrier'] + "_" + train_cat['Dest']

In [ ]:

test_cat = test_.copy()

test_cat['R'] = test_cat['Origin'] + "_" + test_cat['Dest']

test_cat['DepTimeCategory'] = pd.cut(test_cat['DepTime'], bins=[0,500,1200,1700,2100,2400], labels=['Night','Morning', 'Afternoon', 'Evening','Night'], ordered=False)

test_cat['DistanceCategory'] = pd.cut(test_cat['Distance'], bins=[0,1500,4100,5000], labels=['Short-haul','Medium-haul' ,'Long-haul' ])

test_cat.dropna(inplace=True)

test_cat['Month'] = test_cat['Month'].str.split('-').str[-1]
test_cat['Month'] = test_cat['Month'].astype(int)

test_cat['DayofMonth'] = test_cat['DayofMonth'].str.split('-').str[-1]
test_cat['DayofMonth'] = test_cat['DayofMonth'].astype(int)

test_cat['DayOfWeek'] = test_cat['DayOfWeek'].str.split('-').str[-1]
test_cat['DayOfWeek'] = test_cat['DayOfWeek'].astype(int)

test_cat['Carrier+Origin'] = test_cat['UniqueCarrier'] + "_" + test_cat['Origin']
test_cat['Carrier+Dest'] = test_cat['UniqueCarrier'] + "_" + test_cat['Dest']


#Divisão

In [ ]:
X = df_onehot.drop(['Y'],axis=1)
y = df_onehot['Y']

NameError: ignored

In [ ]:

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= SEED)

NameError: ignored

##Divisão para o Catboost

In [ ]:
x_cat = train_cat.drop(['Y'], axis = 1)
y_cat = train_cat['Y']

x_cat_train, x_cat_test, y_cat_train, y_cat_test = train_test_split(x_cat, y_cat, test_size = 0.2, random_state = SEED, stratify = y_cat)

#Árvore de Decisão

In [ ]:
dtmodel = DecisionTreeClassifier(criterion='gini', max_depth = 15)

num_folds = 5


scores = cross_val_score(dtmodel, x_train, y_train, cv=5, scoring='roc_auc')

print("Cross-validation Scores:", scores)
print("Mean roc_auc:", scores.mean())

Cross-validation Scores: [0.66411305 0.67131731 0.67135621 0.67712187 0.66869795]
Mean Accuracy: 0.6705212777259361


In [ ]:

dtmodel.fit(x_train, y_train)
dt_ypred=dtmodel.predict(x_test)
print('roc_auc_score: ', roc_auc_score(y_test, dt_ypred))
print('precision: ', precision_score(y_test, dt_ypred))
print('recall: ', recall_score(y_test, dt_ypred))


roc_auc_score:  0.555277509077171
precision:  0.5253227408142999
recall:  0.14002117522498678


In [ ]:
features = X.columns
dot_data = export_graphviz(dtmodel, feature_names = features, out_file = None, filled = True, rounded = True, class_names= ['nao', 'sim'])
graph = graphviz.Source(dot_data)
graph

#Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, random_state=SEED)

scores = cross_val_score(rf_model, x_train, y_train, cv=5, scoring='roc_auc')

print("Cross-validation Scores:", scores)
print("Mean roc_auc:", scores.mean())

Cross-validation Scores: [0.71391882 0.7154891  0.71559737 0.71968428 0.71580158]
Mean Accuracy: 0.7160982300766985


In [ ]:
rf_model.fit(x_train, y_train)
rf_ypred=rf_model.predict(x_test)

print('roc_auc_score: ', roc_auc_score(y_test, rf_ypred))
print('precision: ', precision_score(y_test, rf_ypred))
print('recall: ', recall_score(y_test, rf_ypred))

roc_auc_score:  0.5387421639625787
precision:  0.5982758620689655
recall:  0.0918475383800953


#Gradient Boosting Decision Tree (GBTD)

##Classifier

In [ ]:
cb_grid = CatBoostClassifier(eval_metric='AUC', cat_features= x_cat.columns.tolist(), early_stopping_rounds=100, learning_rate=0.1, depth=5, l2_leaf_reg=4, rsm=1.0, n_estimators=1000)
cb_grid.fit(x_cat_train, y_cat_train, eval_set=(x_cat_test, y_cat_test), cat_features=x_cat.columns.tolist(), use_best_model=True, verbose=True)

0:	test: 0.6716111	best: 0.6716111 (0)	total: 141ms	remaining: 2m 20s
1:	test: 0.6781415	best: 0.6781415 (1)	total: 293ms	remaining: 2m 26s
2:	test: 0.6867629	best: 0.6867629 (2)	total: 496ms	remaining: 2m 44s
3:	test: 0.6913165	best: 0.6913165 (3)	total: 581ms	remaining: 2m 24s
4:	test: 0.6913462	best: 0.6913462 (4)	total: 731ms	remaining: 2m 25s
5:	test: 0.6907000	best: 0.6913462 (4)	total: 923ms	remaining: 2m 32s
6:	test: 0.6915233	best: 0.6915233 (6)	total: 1.04s	remaining: 2m 26s
7:	test: 0.6927184	best: 0.6927184 (7)	total: 1.19s	remaining: 2m 27s
8:	test: 0.6952342	best: 0.6952342 (8)	total: 1.36s	remaining: 2m 30s
9:	test: 0.7033751	best: 0.7033751 (9)	total: 1.62s	remaining: 2m 40s
10:	test: 0.7063410	best: 0.7063410 (10)	total: 1.83s	remaining: 2m 44s
11:	test: 0.7066705	best: 0.7066705 (11)	total: 2.04s	remaining: 2m 48s
12:	test: 0.7085522	best: 0.7085522 (12)	total: 2.21s	remaining: 2m 48s
13:	test: 0.7117582	best: 0.7117582 (13)	total: 2.41s	remaining: 2m 50s
14:	test: 0.

#Entrega

In [ ]:
gb_test_pred = cb_grid.predict_proba(test_cat)[:, 1]
pd.Series(gb_test_pred, name='dep_delayed_15min').to_csv('popg.csv', index_label='id', header=True)